In [ ]:
from keras.applications import VGG16
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

# Loading data

In [ ]:
path_train="../input/rca-lca/train"
path_test="../input/rca-lca/test"

Image Augmentation

In [ ]:
batch_size=32
nb_epochs=20

In [ ]:
# Create data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(path_train,
                                                    subset='training',
                                                    target_size=(512, 512),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

In [ ]:
validation_generator = train_datagen.flow_from_directory(path_train,
                                                        subset='validation',
                                                        target_size=(512, 512),
                                                        batch_size=batch_size,
                                                        class_mode='binary')

In [ ]:
# Evaluate the model on test data
test_generator = test_datagen.flow_from_directory(path_test,
                                                  target_size=(512, 512),
                                                  batch_size=batch_size,
                                                  class_mode='binary')

In [ ]:
taille_test=1482


# 1- VGG-16

Loading the Base Model

In [ ]:
# Load pre-trained model without the top layer (classifier)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Freeze all layers in the base model so they are not trained during the fine-tuning process
for layer in base_model.layers:
    layer.trainable = False

# Add a new classifier on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model_vgg = Model(inputs=base_model.input, outputs=predictions)

Compile and Fit

In [ ]:
# Compile the model with binary cross-entropy loss and Adam optimizer
model_vgg.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
vgg16_model_name="best_vgg_Model.h5"
checkpoint = ModelCheckpoint(vgg16_model_name,  # model filename
                             monitor='loss',
                             verbose=1, # verbosity - 0 or 1
                             save_best_only= True,
                             mode='auto')
early_stopping = EarlyStopping(monitor='loss',
                               patience=5,
                               verbose=1,
                               mode='auto')

In [ ]:
# Train the model
history_vgg16=model_vgg.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=nb_epochs,
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks=[checkpoint, early_stopping]
        )

In [ ]:
plt.figure(figsize=(10, 40))

for idx, key in enumerate(history_vgg16.history.keys()):

    ax = plt.subplot(8, 2, 1 + idx)
    plt.title(key)
    plt.plot(history_vgg16.history[key])

plt.show()

In [ ]:
# load best model
best_vgg = keras.models.load_model('/kaggle/working/'+vgg16_model_name)
print('loaded...')

In [ ]:
score = best_vgg.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#Confution Matrix and Classification Report
Y_pred = best_vgg.predict(test_generator, taille_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
print('Classification Report')
target_names = ['Left', 'Right']
print(classification_report(test_generator.classes,
                            y_pred,
                            target_names=target_names))

In [ ]:
import pickle
#save history
with open('/kaggle/working/history_vgg16', 'wb') as file_pi:
    pickle.dump(history_vgg16.history, file_pi)
print('saved')

In [ ]:
#load history
with open('/kaggle/working/history', "rb") as file_pi:
    history1 = pickle.load(file_pi)

# 2- Inception

Loading the Base Model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (512, 512, 3),
                         include_top = False,
                         weights = 'imagenet'
                        )

# Freeze all layers in the base model so they are not trained during the fine-tuning process
for layer in base_model.layers:
    layer.trainable = False

# Add a new classifier on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model_incept = Model(inputs=base_model.input, outputs=predictions)

Compile and Fit

In [ ]:
# Compile the model with binary cross-entropy loss and Adam optimizer
model_incept.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
incept_model_name="best_incept_Model.h5"
checkpoint = ModelCheckpoint(incept_model_name,  # model filename
                             monitor='loss',
                             verbose=1, # verbosity - 0 or 1
                             save_best_only= True,
                             mode='auto')
early_stopping = EarlyStopping(monitor='loss',
                               patience=5,
                               verbose=1,
                               mode='auto')

In [ ]:
# Train the model
history_incept=model_incept.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=nb_epochs,
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks=[checkpoint, early_stopping]
        )

In [ ]:
# load best model
best_incept = keras.models.load_model('/kaggle/working/'+incept_model_name)
print('loaded...')

In [ ]:
plt.figure(figsize=(10, 40))

for idx, key in enumerate(history_incept.history.keys()):

    ax = plt.subplot(8, 2, 1 + idx)
    plt.title(key)
    plt.plot(history_incept.history[key])

plt.show()

In [ ]:
score = best_incept.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#Confution Matrix and Classification Report
Y_pred = best_incept.predict(test_generator, taille_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
print('Classification Report')
target_names = ['Left', 'Right']
print(classification_report(test_generator.classes,
                            y_pred,
                            target_names=target_names))

In [ ]:
#save history
with open('/kaggle/working/history_incept', 'wb') as file_pi:
    pickle.dump(history_incept.history, file_pi)
print('saved')

# 3- ResNet50

Import the base model

Build and Compile the Model

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(512, 512,3),
                      include_top=False,
                      weights="imagenet"
                     )

# Freeze all layers in the base model so they are not trained during the fine-tuning process
for layer in base_model.layers:
    layer.trainable = False

# Add a new classifier on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model_resNet = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model with binary cross-entropy loss and Adam optimizer
model_resNet.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
resNet_model_name="best_resNet_Model.h5"
checkpoint = ModelCheckpoint(resNet_model_name,  # model filename
                             monitor='loss',
                             verbose=1, # verbosity - 0 or 1
                             save_best_only= True,
                             mode='auto')
early_stopping = EarlyStopping(monitor='loss',
                               patience=5,
                               verbose=1,
                               mode='auto')

Fitting the model

In [ ]:
# Train the model
history_resNet=model_resNet.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=nb_epochs,
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks=[checkpoint, early_stopping]
        )

In [ ]:
plt.figure(figsize=(10, 40))

for idx, key in enumerate(history_resNet.history.keys()):

    ax = plt.subplot(8, 2, 1 + idx)
    plt.title(key)
    plt.plot(history_resNet.history[key])

plt.show()

In [ ]:
# load best model
best_resNet = keras.models.load_model('/kaggle/working/'+resNet_model_name)
print('loaded...')

In [ ]:
score = best_resNet.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#Confution Matrix and Classification Report
Y_pred = best_resNet.predict(test_generator, taille_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
print('Classification Report')
target_names = ['Left', 'Right']
print(classification_report(test_generator.classes,
                            y_pred,
                            target_names=target_names))

In [ ]:
#save history
with open('/kaggle/working/history_resNet', 'wb') as file_pi:
    pickle.dump(history_resNet.history, file_pi)
print('saved')

# 4- EfficientNet

Loading the Base Model

In [ ]:
!pip install efficientnet

In [ ]:
eff_model_name="best_eff_Model.h5"
checkpoint = ModelCheckpoint(eff_model_name,  # model filename
                             monitor='loss',
                             verbose=1, # verbosity - 0 or 1
                             save_best_only= True,
                             mode='auto')
early_stopping = EarlyStopping(monitor='loss',
                               patience=5,
                               verbose=1,
                               mode='auto')

In [ ]:
import efficientnet.keras as efn
base_model = efn.EfficientNetB0(input_shape = (512, 512, 3),
                                include_top = False,
                                weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Add a new classifier on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model_eff = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model with binary cross-entropy loss and Adam optimizer
model_eff.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model
history_eff=model_eff.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=nb_epochs,
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks=[checkpoint, early_stopping]
        )

In [ ]:
plt.figure(figsize=(10, 40))

for idx, key in enumerate(history_eff.history.keys()):

    ax = plt.subplot(8, 2, 1 + idx)
    plt.title(key)
    plt.plot(history_eff.history[key])

plt.show()

In [ ]:
# load best model
best_effNet = keras.models.load_model('/kaggle/working/'+eff_model_name)
print('loaded...')

In [ ]:
score = best_effNet.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#Confution Matrix and Classification Report
Y_pred = best_effNet.predict(test_generator, taille_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
print('Classification Report')
target_names = ['Left', 'Right']
print(classification_report(test_generator.classes,
                            y_pred,
                            target_names=target_names))

In [ ]:
import os
from pathlib import Path
file='output.zip'
if os.path.exists(Path(file)) :
    os.remove(file)
!zip -r output.zip /kaggle/working

In [ ]:
#save history
with open('/kaggle/working/history_eff', 'wb') as file_pi:
    pickle.dump(history_eff.history, file_pi)
print('saved')